# Projet Lignes de niveau

On importe ces modules par principe de precaution

In [ ]:
import autograd
from autograd import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

 Ceci est le notebook Jupyter contenant le projet de dessin de lignes de niveau sous python de Maud Roux-Salembien et Jérémy Costanzo

Soit f une fonction de $[0,1]^2$ vers $\mathbb{R}$ supposée continuement différentiable.

On cherche les ensembles de la forme : {$(x,y)\in\mathbb{R}^2|f(x,y)=c$} où c $\in$ $\mathbb{R}$

In [ ]:
def f(x,y):
    return(x**2+y**2)
f(1,2)

## <u>Contour simple</u>

### Amorce : 

Soit c $\in$ $\mathbb{R}$

On cherche une condition suffisante sur f(0,0) et f(0,1) pour qu'il existe t $\in$ [0,1] tel que f(0,t) = c

On sait que f est continuement différentiable, donc en particulier f(0,.), qui désigne la fonction $\begin{align*}
  [0,1] &\to \mathbb{R}\\
  t &\mapsto f(0,t).
\end{align*}$
est continue.


Ainsi, si c appartient au segment [min(f(0,0),f(0,1)),max(f(0,0),f(0,1))], le théorème des valeurs intermédiaires nous assure l'existence d'un t $\in$ [0,1] solution de f(0,t) = c.

Si la condition évoquée est satisfaite, on peut chercher numériquement une solution en utilisant la méthode de la dichotomie :

In [ ]:
def find_seed(g,c=0,eps=2**(-26)):
    if (g(0)-c)*(g(1)-c) > 0:
        return None
    else:
        a = 0
        b = 1
        while b-a > eps:
            m = (a+b)/2
            if (g(m)-c)*(g(a)-c)<=0:
                b = m
            else:
                a = m
        t = (a+b)/2
        return t

La méthode des tangentes de Newton converge plus rapidement, mais nécessite plus d'hypothèses, et on ne peut pas savoir à quel point on est éloigné d'une solution, alors que la méthode de la dichotomie ne nécessite que la continuité de g, et permet de majorer la distance à une solution.

### Propagation

Une première approche pour implémenter la fonction simple_contour consisterait à subdiviser [0,1] en segments de taille $\delta$, et d'utiliser find_seed(g,c,eps) où g est la fonction $\begin{align*}
  [0,1] &\to \mathbb{R}\\
  t &\mapsto f(k\delta,t)
\end{align*}$ où k varie de 0 à int(1/$\delta$). Mais cela n'est pas satisfaisant, il faudrait réaliser environ 1/$\delta$ dichotomies, et l'équation f(k$\delta$,t) = c peut admettre plusieurs solutions éloignées les unes des autres.

Pour la suite, nous avons besoin de calculer le gradient de f en un point. Ainsi nous implementons une classe pt qui permet de representer des elements de $\mathbb{R}^2$, donc des points ou des vecteurs.

In [ ]:
tolerance = 1**(-12) #un petit flottant, choisi un peu au pif pour l'instant...

class pt:
    def __init__(self,x=0.0,y=0.0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return('{'+str(self.x)+','+str(self.y)+'}')
        
    def __mul__(a,b):
        if isinstance(b,int) or isinstance(b,float):
            return pt(a.x*b,a.y*b)
        else:
            return pt(a.x*b.x,a.y*b.y)
            
    def __rmul__(a,b):
        if isinstance(a,int) or isinstance(a,float):
            return pt(a*b.x,a*b.y)
        else:
            return a*b
    def __neg__(self):
        return pt(-self.x,-self.y)
        
    def __add__(a,b):
        return pt(a.x+b.x,a.y+b.y)  
    
    def __inv__(self):
        return pt(1/self.x,1/self.y)
    
    def __div__(self,b):
        return (self * (1/b))
    
    def __rdiv__(self,b):
        return pt(b.x/self,b.y/self)
    
    def norme(self):
        return (self.x**2 + self.y**2)**0.5

    def normalized(self):
        return(self*(1/(self.norme())))
        
    def __sub__(self,b):
        return self+ (-b)
        
    def orth(self):
        return pt(-self.y,self.x)

def list_x_y(p):
    x,y = [],[]
    for i in p:
        x.append(i.x)
        y.append(i.y)
    return(np.array(x),np.array(y))
    
def dist(a,b):
    return (a-b).norme()
    
def grad(p=pt()):
    g = autograd.grad
    t = np.r_[g(f,0)(p.x,p.y),g(f,1)(p.x,p.y)]
    return pt(t[0],t[1])

Nous implementons maintenant la fonction ingrid : ingrid(p,grid) renvoie : p $\in$ grid ou grid est en fait un ensemble de la forme [a,b]x[c,d] où (a,b,c,d) $\in \mathbb{R}^4$.

In [ ]:
def ingrid(p,grid):
    return(grid[0]<=p.x<=grid[1] and grid[2]<=p.y<=grid[3])    

Les fonctions suivantes permettent de se reperer sur un cercle de rayon delta centre sur un point p0.

In [ ]:
def p_a1(ang):
    """
    Cette fonction calcule le point p sur le cercle unite tel que l'angle oriente (pt(1,0),p) vale ang
    """
    return(pt(np.cos(ang),np.sin(ang)))    

def rad(p):
    """
    Cette fonction calcule l'angle oriente (pt(1,0),p)
    """
    x = p.x
    y = p.y
    if abs(x)>abs(y):
        if y >= 0:
            return(np.arccos(x))
        else:
            return(-np.arccos(x))
    else:
        if x >= 0:
            return(np.arcsin(y))
        else:
            return(np.pi-np.arcsin(y))
    
def theta(p0,p):
    """
    calcule l'angle oriente (p0+pt(delta,0),p) ou delta = dist(p0,p)
    """
    de = dist(p0,p)
    return(rad((1/de)*(p-p0)))

def p_a(p0,th,delta):
    """
    calcule le point p tel que dist(p,p0) = delta et theta(p0,p) = th
    """
    p = delta*p_a1(th)
    return(p0+p)

Description de suivant, de la methode des tangentes, contour simple...

In [ ]:
def simple_contour(f,c=0.0, delta =0.01,grid = [0.0,1.0,0.0,1.0],cote= lambda x: f(grid[0],x)):
    pass

## Contour complexe

On veut maintenant quadriller notre espace pour trouver des amorces ailleurs pour les lignes de niveau.
On va appeler contour_simple sur ces grilles.
On définit la fonction bord, qui renvoie une fonction $[0,1] \rightarrow \mathbb{R}$, qui va nous servir à trouver une amorce pour chacun des 4 côtés de la cellule.

In [ ]:
def bord(f,p1,p2,n):
                """
                bord(f,p1,p2,n) renvoie une application partielle qui va etre utilisee par find_seed pour trouver une amorce a une ligne de niveau.
                """
                if n == 0:
                    return(lambda x: f(p1.x,p2.y + (p1.y-p2.y)*x))
                elif n == 1:
                    return(lambda x: f(p1.x + (p2.x-p1.x)*x,p1.y))
                elif n == 2:
                    return(lambda t: f(p2.x, p2.y + (p1.y-p2.y)*x))
                elif n == 3:
                    return(lambda t: f(p1.x + (p2.x-p1.x), p2.y))
            
def contour(f,c=0.0,xc=[0.0,1.0],yc=[0.0,1.0],delta = 0.01):
    xs,ys = [],[]
    for i in range(len(xc)-1):
        for j in range(len(yc)-1):
            grid = [xc[i],yc[j],xc[i+1],yc[j+1]]
            p1 = pt(xc[i],yc[i])
            p2 = pt(xc[i+1],yc[j+1])
            
            for n in range(4):    
                appp = bord(f,p1,p2,n)
                x,y = simple_contour(f,c,delta,grid,bord)
                xs.append(x)
                ys.append(y)
    return(xs,ys)